# Parse RT11 Object Files

In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:60% !important; }</style>"))

In [2]:
from rt11_utils import radix50_decode, radix50_encode

## Parse a .OBJ File


In [121]:
word = lambda x: (x).to_bytes(2,"little")
read_word = lambda x: int.from_bytes(x,"little")

def parse_obj_file_formatted_blocks(filename):
    BLOCK_HEADER = b'\x01\x00'
    BLOCK_END = b'\x00\x00'

    output_bytes = []
    al = open(filename,'rb')
    blocks = []
    while True:
        sentinel = al.read(2)
        if sentinel != BLOCK_HEADER and sentinel != BLOCK_END:
            raise Exception("Block Header Expected But Got: ", sentinel)
        if sentinel == BLOCK_END:
            break

        len_data = read_word(al.read(2)) - 4
        data = al.read(len_data)
        checksum = al.read(1)
        output_bytes += [data]
        
    return output_bytes

data = parse_obj_file_formatted_blocks("rk05_tempest_object_files/ALSCO2.OBJ")
[d.hex() for d in data]

['010033082a150000000001af940f08012107e60ee91b40040000e60e477740040000b214477740040000',
 '01001b156927400400006f1afb8040040000081c643440040000e61e4354400400006a1fe00640040000',
 '010077213f5440040000343280254004000071335b4d400400007b335b4d40040000723a256040040000',
 '0100793a7c4c400400003446da4b40040000de4b2021400400007e4c5b4d40040000114e9c5140040000',
 '0100114eae51400400007851d972400400007c511909400400009f516e4c400400009f51145a40040000',
 '01009f51db5a40040000c551655e40040000c5517f5e40040000c551805e40040000c551815e40040000',
 '0100ca51e91f400400001652da7d400400002052da64400400005952ae53400400008952132d40040000',
 '01008f52c84b40040000b6529a77400400002553548a400400007f53eb8a40040000cc53280a40040000',
 '0100cc533a0a40040000cc53309f40040000d2534b2240040000d253fa2640040000d253ae5340040000',
 '010011543859400400001154c87d400400003f54634b400400004854fd39400400004d54116840040000',
 '0100695408524004000047564d084004000039775b4d400400003d772d15400400004477b52240040000',
 '01004777ce0f4004000

In [125]:

def gsd_block(block_code,data):
    """
    Global Symbol Directory Block
    """
    print("Global Symbol Directory Block", f"{block_code:#0{4}x}")
    i = 0
    while i < len(data):
        print("\tData Block Name", radix50_decode(read_word(data[i:i+2])) + radix50_decode(read_word(data[i+2:i+4])))
        i+=4
        print("\tFlags", f"{data[i]:#0{4}x}")
        i+=1
        print("\tEntry Type", f"{data[i]:#0{4}x}")
        i+=1
        print("\tValue", data[i:i+2].hex())
        i+=2

def end_gsd_block(block_code,data):
    print("End Global Symbol Directory Block", f"{block_code:#0{4}x}", data.hex())

def global_additive_relocation(data):
    print("\tGlobal Additive Relocation")
    print("\t\tSymbol Name", radix50_decode(read_word(data[:2])) + radix50_decode(read_word(data[2:4])))
    print("\t\tConstant", read_word(data[4:6]))
    return 6

def internal_relocation(data):
    print("\tInternal Relocation")
    print("\t\tConstant", data[:2].hex())
    return 2

def global_relocation(data):
    print("\tGlobal Relocation")
    print("\t\tSymbol Name", radix50_decode(read_word(data[:2])) + radix50_decode(read_word(data[2:4])))
    return 4

def internal_displaced_relocation(data):
    print("\tInternal Displaced Relocation")
    print("\t\tConstant", data[:2].hex())
    return 2

def global_displaced_relocation(data):
    print("\tGlobal Displaced Relocation")
    print("\t\tSymbol Name", radix50_decode(read_word(data[:2])) + radix50_decode(read_word(data[2:4])))
    return 4

def global_additive_displaced_relocation(data):
    print("\tGlobal Additive Displaced Relocation")
    print("\t\tSymbol Name", radix50_decode(read_word(data[:2])) + radix50_decode(read_word(data[2:4])))
    print("\t\tConstant", data[4:6].hex())
    return 6

def location_counter_modification(data):
    print("\tLocation Counter Modification")
    print("\t\tConstant", data[:2].hex())
    return 2

# RLD Block Commands
rld_block_entry_types = {
    0x001: internal_relocation,
    0x002: global_relocation,
    0x003: internal_displaced_relocation,
    0x004: global_displaced_relocation,
    0x005: global_additive_relocation,
    0x006: global_additive_displaced_relocation,
    0x008: location_counter_modification,
}

def rld_block(block_code,data):
    """
    Relocation Directory Information
    """
    print("Relocation Directory Information", f"{block_code:#0{4}x}")
    i = 0
    while i < len(data):
        high_byte = data[i] & 0x80
        command = data[i] & 0x7F
        print("\tHigh Byte", f"{high_byte:#0{4}x}")
        print("\tCommand", f"{command:#0{4}x}")
        i+=1
        if command not in rld_block_entry_types:
            print("\tUnformatted Value", data[i:].hex())
            return 
        print("\tDisplacement", f"{data[i]:#0{4}x}")
        i+=1
        i += rld_block_entry_types[command](data[i:])

def txt_block(block_code,data):
    """
    Text Information
    """
    print("Text Information", f"{block_code:#0{4}x}")
    print("\tLoad Address", data[:2].hex())
    print("\tText", data[2:].hex())

def end_module(block_code,data):
    print("End Module Block", f"{block_code:#0{4}x}", data.hex())

block_map = {
    0x001: gsd_block,
    0x002: end_gsd_block,
    0x003: txt_block,
    0x004: rld_block,
    0x006: end_module,
}
for block in data:
    block_code = read_word(block[:2])
    if block_code not in block_map:
        raise Exception("Unrecognised Block Type", f"{block_code:#0{4}x}")
    block_map[block_code](block_code, block[2:])


Global Symbol Directory Block 0x01
	Data Block Name ALSCOR
	Flags 0x00
	Entry Type 0x00
	Value 0000
	Data Block Name . ABS.
	Flags 0x08
	Entry Type 0x01
	Value 2107
	Data Block Name BONDRY
	Flags 0x40
	Entry Type 0x04
	Value 0000
	Data Block Name BONSCO
	Flags 0x40
	Entry Type 0x04
	Value 0000
	Data Block Name CLRSCO
	Flags 0x40
	Entry Type 0x04
	Value 0000
Global Symbol Directory Block 0x01
	Data Block Name COCFLI
	Flags 0x40
	Entry Type 0x04
	Value 0000
	Data Block Name DIGTYS
	Flags 0x40
	Entry Type 0x04
	Value 0000
	Data Block Name DSPHOL
	Flags 0x40
	Entry Type 0x04
	Value 0000
	Data Block Name D70MSK
	Flags 0x40
	Entry Type 0x04
	Value 0000
	Data Block Name EABAD 
	Flags 0x40
	Entry Type 0x04
	Value 0000
Global Symbol Directory Block 0x01
	Data Block Name ENGMSG
	Flags 0x40
	Entry Type 0x04
	Value 0000
	Data Block Name HALF  
	Flags 0x40
	Entry Type 0x04
	Value 0000
	Data Block Name HIILOC
	Flags 0x40
	Entry Type 0x04
	Value 0000
	Data Block Name HISLOC
	Flags 0x40
	Entry Type 0x

	Load Address 1007
	Text 04
Relocation Directory Information 0x04
	High Byte 0x00
	Command 0x08
	Displacement 0x00
	Location Counter Modification
		Constant 1507
Text Information 0x03
	Load Address 1507
	Text b85000
Relocation Directory Information 0x04
	High Byte 0x00
	Command 0x08
	Displacement 0x00
	Location Counter Modification
		Constant 0a07
Text Information 0x03
	Load Address 0a07
	Text 0d
Relocation Directory Information 0x04
	High Byte 0x00
	Command 0x08
	Displacement 0x00
	Location Counter Modification
		Constant 1807
Text Information 0x03
	Load Address 1807
	Text a57ccd2701f002b00038ed0002d00018
Relocation Directory Information 0x04
	High Byte 0x00
	Command 0x08
	Displacement 0x00
	Location Counter Modification
		Constant 2607
Text Information 0x03
	Load Address 2607
	Text 01
Relocation Directory Information 0x04
	High Byte 0x00
	Command 0x08
	Displacement 0x00
	Location Counter Modification
		Constant 2807
Text Information 0x03
	Load Address 2807
	Text b000e67be67c
Relocati

In [127]:
hex(0xaf10-0xa8e5)

'0x62b'